# Acceso a la Data

## Generar Token

In [16]:
import os
import base64
import json
import requests as rq
from dotenv import load_dotenv

def obtener_token_idealista(numero_api=1, env_path=None):
    """
    Obtiene un token de acceso de la API de Idealista.
    
    Parámetros:
    - numero_api (int): número de conjunto de claves (1, 2, ...)
    - env_path (str): ruta absoluta al archivo .env (si no se pasa, usa uno por defecto)
    
    Retorna:
    - token de autenticación como string
    """
    
    if env_path is None:
        env_path = r"C:\Users\nicol\Documents\Data_Science\Projects\Project_1\.env"
    
    # Cargar las variables desde el archivo .env
    load_dotenv(dotenv_path=env_path)
    
    # Construir nombres de clave
    api_key_name = f"API_KEY_{numero_api}"
    secret_name = f"API_SECRET_{numero_api}"
    
    # Obtener valores del entorno
    api_key = os.getenv(api_key_name)
    secret = os.getenv(secret_name)
    
    if not api_key or not secret:
        raise ValueError(f"No se encontraron las claves {api_key_name} y/o {secret_name} en el archivo .env")
    
    # Codificar credenciales
    message = f"{api_key}:{secret}"
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")

    headers_dic = {
        "Authorization": auth,
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }

    params_dic = {
        "grant_type": "client_credentials",
        "scope": "read"
    }

    # Enviar solicitud
    response = rq.post(
        "https://api.idealista.com/oauth/token",
        headers=headers_dic,
        params=params_dic
    )

    try:
        token = json.loads(response.text)['access_token']
        return token
    except Exception as e:
        print("❌ Error al obtener el token:", e)
        print("Respuesta completa:")
        print(response.text)
        return None


In [ ]:
# Obtener token de la primera clave
token1 = obtener_token_idealista(1)
print("Token 1:", token1)

# Obtener token de la segunda clave
token2 = obtener_token_idealista(2)
print("Token 2:", token2)

## Filtro para la busqueda

In [18]:
# Parámetros generales de búsqueda
base_url = 'https://api.idealista.com/3.5/'  # Base search url
country = 'es'  # Country code
language = 'es'  # Language
max_items = '50'  # Máximo permitido por llamada
operation = 'sale'  # Tipo de operación: venta
property_type = 'homes'  # Tipo de propiedad
order = 'publicationDate'  # Ordenar por fecha de publicación
sort = 'desc'  # Más recientes primero
bankOffer = 'false'  # No filtrar por bancos
maxprice = '850000'
minprice = '90000'

# Diccionario de ubicaciones
location_ids = {
    'madrid': '0-EU-ES-28',
    'barcelona': '0-EU-ES-08'
}

# Función para definir URL por ciudad
def define_search_url(location_name,pagination=1):
    '''
    Crea la URL de búsqueda personalizada para una ciudad específica.
    location_name: "madrid" o "barcelona"
    pagination: número de página (por defecto = 1)
    '''
    location_id = location_ids.get(location_name.lower())
    if not location_id:
        raise ValueError(f"Ubicación no válida: {location_name}")

    url = (
        base_url +
        country +
        '/search?operation=' + operation +
        '&maxItems=' + max_items +
        '&order=' + order +
        '&locationId=' + location_id +
        '&propertyType=' + property_type +
        '&sort=' + sort +
        '&numPage=' + str(pagination) +
        '&maxPrice=' + maxprice +
        '&minPrice=' + minprice +
        '&language=' + language
    )
    return url


In [ ]:
# URL generado para la busqueda
madrid_url = define_search_url('madrid')
barcelona_url = define_search_url('barcelona')

print("URL Madrid:", madrid_url)
print("URL Barcelona:", barcelona_url)

## Busqueda de datos

In [20]:
# Función para hacer la busqueda
def search_api(token, url):
    '''
    This function will use the token and url created previously, and return our search results.
    '''
    headers = {'Content-Type': 'Content-Type: multipart/form-data;',   # Define the search headers
                'Authorization' : 'Bearer ' + token}

    content = rq.post(url, headers = headers)   # Return the content from the request

    result = json.loads(content.text)   # Transform the result as a json file

    return result


In [21]:
first_search = search_api(token1,barcelona_url)

In [ ]:
# Cantidad de paginas 
total_pages=first_search
print(total_pages['totalPages'])

In [3]:
import pandas as pd

def results_to_df(results):
    '''
    This function will save the json results as a dataframe and return the resulting dataframe
    '''
    df = pd.DataFrame.from_dict(results['elementList'])

    return df

def concat_df(df, df_tot):
    '''
    This function will take the main dataframe (df_tot), and concat it with the given individual dataframe,
    returning the main dataframe
    '''
    df_tot= pd.concat([df_tot,df],ignore_index=True)

    return df_tot

def df_to_csv(df,file_path):
    '''
    This function will take a given dataframe and save it as a csv file
    '''
    df = df.reset_index()   # Reset the index in order to organise the records
    df.to_csv(file_path, index=False)   # Save it into a csv

In [26]:
df = results_to_df(first_search)
df_tot = pd.DataFrame()
df_tot = concat_df(df, df_tot)

In [ ]:
# Getting the data 
for i in range(2, 200):
    url = define_search_url('barcelona',i) # Add the pagination to the url
    results = search_api(token1,url)   # Get the search results
    df = results_to_df(results)   # Save the results as a dataframe
    df_tot = concat_df(df, df_tot)   # Concat the results to the main dataframe

In [ ]:
from datetime import datetime
# Obtener fecha y hora actual
ahora = datetime.now()
# Formatear: Idealista_YYYY-MM-DD_HH-MM-SS
name_file_dt=fr"idealista_{ahora.strftime('%Y-%m-%d_%H-%M-%S')}.csv"
name_file_=fr"idealista_bcn_sale_08_2025.csv"

file_path=fr'..\data\+{name_file}'

df_to_csv(df_tot,file_path)

## Visualizacion de los datos

In [ ]:
# Ruta relativa
df_idealista_bcn = pd.read_csv(r"..\data\idealista_bcn_sale_08_2025.csv")
print(df_idealista_bcn.info())